# StockNewsDeepProvider

**stock_news_deep_provider.ipynb**

A tool-using research agent that answers the user query in our case (“Should I buy Apple stock?”) by:

1. Defining a search tool (ddg_search) that wraps a DuckDuckGo client.
2. Setting a system role (rsch_instructions) that frames the model as an expert financial researcher and documents how to use the search tool.
3. Building the agent with create_deep_agent so the model can call the tool autonomously.
4. Invoking the agent with a user query.

Why StockNewsDeepProvider matters ?

Upgrades chatbot to researcher using live web search instead of guessing from pretraining which makes it more transparent, customizable, and focused on local data. With Ollama, you run the reasoning model locally (cost control, latency, privacy), while still reaching the web through a controlled tool. This is the core agentic loop that you can reuse for various tasks, like earnings analysis, competitor scans, and macro briefings (just needs more tools, sub-agents, etc).

In [3]:
from langchain.chat_models import init_chat_model
import os
from typing import Literal
from deepagents import create_deep_agent
from langchain_community.tools import DuckDuckGoSearchResults

researcher = DuckDuckGoSearchResults()

# Ddg search tool
def ddg_search(
    query: str,
    max_results: int = 10,
    category: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Execute a duckduckgo internet search"""
    return researcher.invoke(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=category,
    )


# System prompt
rsch_instructions = """You are an expert financial researcher. Your job is to conduct thorough research, and then write a polished report about a given stock.

You have access to an internet search tool as your primary means of gathering information.

## `ddg_search`

Use this to run an internet search for a given query. You can specify the max number of results to return, the topic, and whether raw content should be included.
"""

model = init_chat_model(
    model="ollama:llama3.2",
)

# Create the deep agent
agent = create_deep_agent(
    model=model,
    tools=[ddg_search],
    system_prompt=rsch_instructions,
)

# Start the agent
agent_result = agent.invoke({"messages": [{"role": "user", "content": "Should i buy Apple Stock?"}]})

In [4]:
# Result printing
for message in agent_result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Should i buy Apple Stock?
================================== Ai Message ==================================
Tool Calls:
  ddg_search (b9769ba9-381d-4445-ad92-ddc10eb28462)
 Call ID: b9769ba9-381d-4445-ad92-ddc10eb28462
  Args:
    category: general
    include_raw_content: False
    max_results: 1
    query: should I buy Apple stock
================================= Tool Message =================================
Name: ddg_search

snippet: ... Apple stock is supported by healthy free cash flow, a dividend of 90 ... 00:00 Should you buy Apple stock ? Apple has a market cap of $2.39 trillion., title: Should you buy Apple stock? March 2023 - video Dailymotion, link: https://www.dailymotion.com/video/x8ppktq, snippet: ... www.overlookedalpha.com So far in 2022, Apple stock has ... 00:00 Should you buy Apple stock ? So far in 2022 Apple stock has outperformed its rivals., title: Should you buy Apple stock? 3-min